# Part I. ETL Pipeline for Pre-Processing the Files

#### RUN THE FOLLOWING to PRE-PROCESS Input FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Creating a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

## The CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory is ready to be used.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [2]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [3]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [4]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

#### Now we need to create tables to run the following queries. With Apache Cassandra, we model the database tables on the queries we want to run.

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'



### Query1:
#### Query Description:
"Song_Details_Based_On_Session_And_Item" table is modeled to fetch song details. The data is partitioned using the value of partition key which in this case is a unique combination of both sessionId and itemInSession. Partition key helps to identify the node/cluster that holds the data while avoiding the search across all the other nodes. This makes the query efficient with little turnaround time. 

In [5]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
create_query1 = "CREATE TABLE IF NOT EXISTS Song_Details_Based_On_Session_And_Item "
create_query1 = create_query1 + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY ((sessionId, itemInSession)))"

try:
    session.execute(create_query1)
except Exception as e:
    print(e)
    

In [6]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_query1 = "INSERT INTO Song_Details_Based_On_Session_And_Item (sessionId, itemInSession, artist, song, length)"
        insert_query1 = insert_query1 + " VALUES (%s, %s, %s, %s, %s)"
        try:
            session.execute(insert_query1, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print(e)

#### Below is a SELECT query to verify that data has been inserted into the table

In [7]:
## Add in the SELECT statement to verify the data was entered into the table
select_query1 = "select artist, song, length from Song_Details_Based_On_Session_And_Item where sessionId=338 and itemInSession=4"
try:
    rows = session.execute(select_query1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query2:
#### Query Description:
"Song_Details_Based_On_UserID_And_Session" table is modeled to fetch song details. The data is partitioned using the value of partition key which in this case is a unique combination of both userId and sessionId. The clustering key itemInSession helps to sort rows based on the value of the column and in ASC order. 

In [8]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
create_query2 = "CREATE TABLE IF NOT EXISTS Song_Details_Based_On_UserID_And_Session "
create_query2 = create_query2 + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"

try:
    session.execute(create_query2)
except Exception as e:
    print(e)                    

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_query2 = "INSERT INTO Song_Details_Based_On_UserID_And_Session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        insert_query2 = insert_query2 + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        try:
            session.execute(insert_query2, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        except Exception as e:
            print(e)

#### Below is a SELECT query to verify that data has been inserted into the table

In [19]:
select_query2 = "select artist, song, firstName, lastName from Song_Details_Based_On_UserID_And_Session where userId=10 and sessionId=182"

try:
    rows = session.execute(select_query2)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query3:
#### Query Description:
"User_Details_Based_On_Song" table is modeled to fetch user details. Song title is used as partition key and userId is used as clustering key. If we use just the partition key (song titile) with no clustering key, although Cassandra would fetch all records with the song title, it would overwrites the duplicate rows and the result would just be 1 row. To avoid this, a clustering key (userId) is also used to help uniquely identify each song with associated user.

In [33]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
create_query3 = "CREATE TABLE IF NOT EXISTS User_Details_Based_On_Song "
create_query3 = create_query3 + "(song text, userId int, firstName text, lastName text, PRIMARY KEY ((song), userId))"

try:
    session.execute(create_query3)
except Exception as e:
    print(e)

                    

In [35]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_query3 = "INSERT INTO User_Details_Based_On_Song (song, userId, firstName, lastName)"
        insert_query3 = insert_query3 + " VALUES (%s, %s, %s, %s)"
        try:
            session.execute(insert_query3, (line[9], int(line[10]), line[1], line[4]))
        except Exception as e:
            print(e)

#### Below is a SELECT query to verify that data has been inserted into the table

In [36]:
select_query3 = "select firstName, lastName from User_Details_Based_On_Song where song='All Hands Against His Own'"
try:
    rows = session.execute(select_query3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [38]:
drop_query1 = "drop table Song_Details_Based_On_Session_And_Item"
drop_query2 = "drop table Song_Details_Based_On_UserID_And_Session"
drop_query3 = "drop table User_Details_Based_On_Song"

try:
    session.execute(drop_query1)
    session.execute(drop_query2)
    session.execute(drop_query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [39]:
session.shutdown()
cluster.shutdown()